In [32]:
# Import Spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd
import string, re, json

# Import NLTK
import nltk
import sys
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/emanuele/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [33]:
ENGRetweet = "./ENGRetweet.json"
ENGNoRetweet = "./ENGNoRetweet.json"

# Creazione dei dataFrame
df_ENGRetweet = spark.read\
    .format("json")\
    .option("inferSchema", "true")\
    .option("multiLine", "true")\
    .load(ENGRetweet)

df_ENGNoRetweet = spark.read\
    .format("json")\
    .option("inferSchema", "true")\
    .option("multiLine", "true")\
    .load(ENGNoRetweet)

In [34]:
# UNIONE 
df_Tweets = df_ENGRetweet\
    .selectExpr("id_str", "retweeted_status.full_text as full_text")\
    .union(df_ENGNoRetweet.select("id_str", "full_text"))


df_Tweets = df_Tweets.select("full_text").distinct()
df_Tweets.count()

1050

In [49]:
def vaderSentimentAnalysis(data_str):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(data_str)
    return ss
vaderSentimentAnalysis_udf = udf(vaderSentimentAnalysis, StringType())

In [52]:
df_Tweets = df_Tweets.withColumn("score", vaderSentimentAnalysis_udf(df_Tweets['full_text']))
df_Tweets.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                         full_text|                                             score|
+--------------------------------------------------+--------------------------------------------------+
|"The United States, long accustomed to thinking...|  {neg=0.0, pos=0.299, compound=0.9076, neu=0.701}|
|Coronavirus &amp; climate change demand similar...|  {neg=0.031, pos=0.25, compound=0.8639, neu=0.72}|
|Oh to be a 1998 baby

✔️ first memory is 9/11

...| {neg=0.233, pos=0.0, compound=-0.7184, neu=0.767}|
|Hookah should be banned at groove and restauran...|     {neg=0.2, pos=0.0, compound=-0.4588, neu=0.8}|
|@caitrionambalfe @netflix My husband and I have...|         {neg=0.0, pos=0.0, compound=0.0, neu=1.0}|
|tangled will be THE movie of self-isolation #CO...|         {neg=0.0, pos=0.0, compound=0.0, neu=1.0}|
|#Epeeps Please take care  🦠🦠🦠🔥🔥🔥  #COVID1...|  {neg=0.0, pos=0.